In [1]:
!pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.44.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.44.2
Y
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_mjlvruc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_mjlvruc
  Resolved https://github.com/huggingface/transformers to commit 2e24ee4dfa39cc0bc264b89edbccc373c8337086
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9922875 sha256=72cb63fc33fff0f09bdc3671a2fe9

In [2]:
pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 63.7 MB/s eta 0:00:00


In [3]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [4]:
%%writefile app.py
import streamlit as st
from huggingface_hub import notebook_login
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch
import re

@st.cache_resource
def load_models():
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2-VL-2B-Instruct",
        torch_dtype=torch.float16,
        device_map="auto",
    )
    min_pixels = 256*28*28
    max_pixels = 1280*28*28
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)
    return model, processor

model, processor = load_models()

st.title("Image Search and Text Extraction App")

uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    temp_image_path = "uploaded_image.png"
    image.save(temp_image_path)

    # Text extraction
    if st.button("Extract Text from Image"):
        st.image(image, caption="Extracting text...", use_column_width=True)

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": temp_image_path},
                    {"type": "text", "text": "extract text"}
                ]
            }
        ]

        text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        image_inputs, video_inputs = process_vision_info(messages)

        inputs = processor(
            text=[text_prompt],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt"
        )

        inputs = inputs.to(model.device)

        # Inference: Generate the output
        generated_ids = model.generate(**inputs, max_new_tokens=2048)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        extracted_text = " ".join(output_text)

        # Store extracted text in Streamlit session state
        st.session_state.extracted_text = extracted_text

        st.subheader("Extracted Text:")
        st.text_area("Extracted Text", extracted_text, height=300)

# Display text query input and perform search if text is extracted
if 'extracted_text' in st.session_state:
    text_query = st.text_input("Enter your text query")

    if st.button("Search Text"):
        if text_query:
            # Highlight the queried text
            def highlight_text(text, query):
                highlighted_text = text
                for word in query.split():
                    pattern = re.compile(re.escape(word), re.IGNORECASE)
                    highlighted_text = pattern.sub(lambda m: f'<span style="background-color: yellow; color: black;">{m.group()}</span>', highlighted_text)
                return highlighted_text

            highlighted_output = highlight_text(st.session_state.extracted_text, text_query)

            st.subheader("Search Results (with query highlighted):")
            st.markdown(highlighted_output, unsafe_allow_html=True)
        else:
            st.warning("Please enter a query.")
else:
    st.info("Upload an image and extract text to get started.")

Writing app.py


In [5]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [6]:
!curl ipv4.icanhazip.com

34.124.173.252


In [7]:
!streamlit run app.py --server.enableXsrfProtection false &>/content/logs.txt & npx localtunnel --port 8501

your url is: https://odd-rooms-count.loca.lt
^C
